This ipynb did the following tasks:

Check vcf data

Parse column 8 into 9 additional columns

Split data into different categories based on their format and save to a new .tsv file

Summarize number of variants in each category

In [ ]:
import pysam

def print_first_5_rows(vcf_path):
    # Open VCF file
    vcf = pysam.VariantFile(vcf_path)
    # Print the header
    print(vcf.header)
    # Print first 5 records
    print("\nFirst 5 variants:")
    for i, record in enumerate(vcf):
        if i >= 5:
            break
        print(record)
    vcf.close()

# Example usage
print_first_5_rows("/share/yu/mw2243/HGMD/2025_1_hg38_fullinfo.vcf")

In [ ]:
import pysam
import pandas as pd
from typing import Dict, Optional

def parse_info_field(info_obj) -> Dict[str, Optional[str]]:
    """Parse VariantRecordInfo object with robust error handling"""
    parsed = {
        'CLASS': None,
        'MUT': None,
        'GENE': None,
        'STRAND': None,
        'DNA': None,
        'PROT': None,
        'DB': None,
        'PHEN': None,
        'RANKSCORE': None
    }
    try:
        # Get all available fields
        available_fields = list(info_obj.keys())
        #print(f"Available INFO fields in first record: {available_fields}")
        # Flexible field name mapping
        field_mapping = {
            'CLASS': ['CLASS', 'CLS', 'TYPE'],
            'MUT': ['MUT', 'MUTATION', 'MUT_TYPE'],
            'GENE': ['GENE', 'GENENAME', 'G'],
            'STRAND': ['STRAND', 'STR'],
            'DNA': ['DNA', 'TRANSCRIPT', 'CDNA'],
            'PROT': ['PROT', 'PROTEIN', 'AA'],
            'DB': ['DB', 'RS', 'DBSNP', 'RSID'],
            'PHEN': ['PHEN', 'PHENOTYPE', 'DISEASE'],
            'RANKSCORE': ['RANKSCORE', 'RANK', 'SCORE']
        }
        for target_field, possible_names in field_mapping.items():
            for name in possible_names:
                if name in info_obj:
                    value = info_obj[name]
                    if isinstance(value, (list, tuple)):
                        value = ','.join(map(str, value))
                    if value:
                        value = str(value).replace('"', '')
                        if target_field in ['DNA', 'PROT']:
                            value = str(value).replace('%3A', ':').replace('%2C', ',')
                    parsed[target_field] = str(value) if value else None
                    break
    except Exception as e:
        print(f"Error parsing INFO: {e}")
    return parsed


def process_vcf(input_vcf: str, output_tsv: str):
    """Process VCF with proper error handling"""
    try:
        # Open VCF without require_index parameter
        vcf = pysam.VariantFile(input_vcf)
        results = []
        for i, record in enumerate(vcf):
            # Basic variant info
            variant_data = {
                'CHROM': record.chrom,
                'POS': record.pos,
                'ID': record.id if record.id else '.',
                'REF': record.ref,
                'ALT': ','.join(record.alts) if record.alts else '.',
                'QUAL': record.qual,
                'FILTER': ','.join(record.filter) if record.filter else 'PASS'
            }
            # Parse INFO
            parsed_info = parse_info_field(record.info)
            results.append({**variant_data, **parsed_info})
            # Debug output for first record
            if i == 0:
                print("\nFirst record INFO content:")
                for k, v in record.info.items():
                    print(f"{k}: {v} ({type(v)})")
        # Create DataFrame
        df = pd.DataFrame(results)
        # Save to TSV
        df.to_csv(output_tsv, sep='\t', index=False)
        print(f"\nSuccessfully processed {len(df)} records")
        print("Sample output:")
        print(df.head().to_string(index=False))
    except Exception as e:
        print(f"Fatal error: {e}")
    finally:
        if 'vcf' in locals():
            vcf.close()
    return df


# Process your file
parsed_res = process_vcf(
    "/share/yu/mw2243/HGMD/2025_1_hg38_fullinfo.vcf",
    "/share/yu/mw2243/HGMD/parsed_variants.tsv"
)

In [ ]:
import pysam

def count_vcf_records(vcf_path):
    with pysam.VariantFile(vcf_path) as vcf:
        count = sum(1 for _ in vcf)
    return count

num_records = count_vcf_records("/share/yu/mw2243/HGMD/2025_1_hg38_fullinfo.vcf")
print(f"Total records: {num_records}")

In [ ]:
import pandas as pd
import re

def extract_missense_variants(input_tsv, output_tsv):
    # Read the parsed TSV file
    df = pd.read_csv(input_tsv, sep='\t')
    # Define the missense mutation pattern
    missense_pattern = r'.+:p\.([A-Z])(\d+)([A-Z])'
    # Create mask for rows with valid missense mutations
    missense_mask = df['PROT'].str.contains(missense_pattern, na=False)
    # Subset the dataframe
    missense_df = df[missense_mask].copy()
    # Extract amino acid information
    aa_info = missense_df['PROT'].str.extract(missense_pattern)
    # Add new columns
    missense_df['REF_AA'] = aa_info[0]  # Reference amino acid
    missense_df['ALT_AA'] = aa_info[2]  # Alternate amino acid
    missense_df['POS_AA'] = aa_info[1]  # Amino acid position (optional)
    # Save to new TSV
    missense_df.to_csv(output_tsv, sep='\t', index=False)
    print(f"Found {len(missense_df)} missense variants")
    print("Sample output:")
    print(missense_df[['PROT', 'REF_AA', 'ALT_AA']].head())
    return missense_df


# Example usage
input_file = "/share/yu/mw2243/HGMD/parsed_variants.tsv"
output_file = "/share/yu/mw2243/HGMD/missense_variants.tsv"
result_df = extract_missense_variants(input_file, output_file)

In [ ]:
import pandas as pd
import re

def extract_nonsense_variants(input_tsv, output_tsv):
    # Read the parsed TSV file
    df = pd.read_csv(input_tsv, sep='\t')
    # Define the missense mutation pattern
    nonsense_pattern = r'.+:p.[A-Z]\d+\*'
    # Create mask for rows with valid missense mutations
    nonsense_mask = df['PROT'].str.contains(nonsense_pattern, na=False)
    # Subset the dataframe
    nonsense_df = df[nonsense_mask].copy()
    # Save to new TSV
    #missense_df.to_csv(output_tsv, sep='\t', index=False)
    print(f"Found {len(nonsense_df)} nonsense variants")
    print("Sample output:")
    #print(missense_df[['PROT', 'REF_AA', 'ALT_AA']].head())
    return nonsense_df

input_file = "/share/yu/mw2243/HGMD/parsed_variants.tsv"
output_file = "/share/yu/mw2243/HGMD/nonsense_variants.tsv"
result_df = extract_nonsense_variants(input_file, output_file)
result_df.to_csv(output_file,sep='\t',index=False)

In [ ]:
#coding indels
coding_indels = parsed_res[
    (parsed_res['REF'].str.len() != parsed_res['ALT'].str.len()) & 
    (parsed_res['PROT'].values!=None)
]
coding_indels.to_csv("/share/yu/mw2243/HGMD/coding_indels.tsv", sep='\t', index=False)

noncoding = parsed_res[
    (parsed_res['PROT'].values==None)
]
noncoding.to_csv("/share/yu/mw2243/HGMD/non_coding.tsv", sep='\t', index=False)

In [ ]:
import pandas as pd

# Read the main parsed variants file
main_df = pd.read_csv("/share/yu/mw2243/HGMD/parsed_variants.tsv", sep="\t")

# Initialize a set to store all IDs from the other files
known_ids = set()

# List of files containing known variant types
known_files = [
    "/share/yu/mw2243/HGMD/non_coding.tsv",
    "/share/yu/mw2243/HGMD/missense_variants.tsv",
    "/share/yu/mw2243/HGMD/nonsense_variants.tsv",
    "/share/yu/mw2243/HGMD/coding_indels.tsv"
]

# Collect all IDs from the known files
for file in known_files:
    try:
        df = pd.read_csv(file, sep="\t")
        if 'ID' in df.columns:
            known_ids.update(df['ID'].dropna().astype(str))
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Filter main dataframe for variants not in any known category
other_variants = main_df[~main_df['ID'].isin(known_ids)]

# Save to new file
other_variants.to_csv("/share/yu/mw2243/HGMD/other_variants.tsv", sep="\t", index=False)
print(f"Saved {len(other_variants)} other variants to /share/yu/mw2243/HGMD/other_variants.tsv")
print(f"Total original variants: {len(main_df)}")
print(f"Known variants: {len(known_ids)}")

In [ ]:
# read in: "/share/yu/mw2243/HGMD/parsed_variants.tsv"
# based on ID of the following files,
# "/share/yu/mw2243/HGMD/non_coding.tsv"
# "/share/yu/mw2243/HGMD/missense_variants.tsv"
# "/share/yu/mw2243/HGMD/nonsense_variants.tsv"
# "/share/yu/mw2243/HGMD/coding_indels.tsv"
# save rest rows to /share/yu/mw2243/HGMD/other_variants.txt

In [ ]:
# missense 
# non-sense mutation: r'.+:p.[A-Z]\d+\*'
# coding indels: From the parsed data frame, pick out variants whose REF_N and ALT_N have different length and RefSeq_protein_record is not nan.
# noncoding variants
# remaining variants

In [ ]:
#count frequency
import pandas as pd

# Dictionary to store counts
category_counts = {}

# List of files with their categories
variant_files = {
    "non_coding": "/share/yu/mw2243/HGMD/non_coding.tsv",
    "missense": "/share/yu/mw2243/HGMD/missense_variants.tsv",
    "nonsense": "/share/yu/mw2243/HGMD/nonsense_variants.tsv", 
    "coding_indels": "/share/yu/mw2243/HGMD/coding_indels.tsv",
    "other_variants": "/share/yu/mw2243/HGMD/other_variants.tsv"
}

# Count rows in each file
for category, filepath in variant_files.items():
    try:
        # Only read first column to be memory efficient
        df = pd.read_csv(filepath, sep='\t', usecols=[0]) 
        category_counts[category] = len(df)
    except Exception as e:
        print(f"Error counting {category}: {e}")
        category_counts[category] = 0

# Print results        
print("Variant Category Counts:")
for category, count in category_counts.items():
    print(f"{category:>15}: {count:>6,} rows")

# Calculate and print total
total = sum(category_counts.values())
print(f"{'Total':>15}: {total:>6,} rows")

In [ ]:
category_counts = {}
variant_files = {
    "mapped coding_indels":"/share/yu/mw2243/HGMD/processed/mapped_coding_indel.txt",
    "mapped missense":"/share/yu/mw2243/HGMD/processed/mapped_missense.txt",
    "mapped nonsense":"/share/yu/mw2243/HGMD/processed/mapped_nonsense.txt",
    "non_coding": "/share/yu/mw2243/HGMD/processed/non_coding.tsv",
    "other variants":"/share/yu/mw2243/HGMD/processed/other_variants.tsv",
    "unmapped coding_indels":"/share/yu/mw2243/HGMD/processed/unmapped_coding_indel.txt",
    "unmapped missense":"/share/yu/mw2243/HGMD/processed/unmapped_missense.txt",
    "unmapped nonsense":"/share/yu/mw2243/HGMD/processed/unmapped_nonsense.txt"
}

# Count rows in each file
for category, filepath in variant_files.items():
    try:
        # Only read first column to be memory efficient
        df = pd.read_csv(filepath, sep='\t', usecols=[0]) 
        category_counts[category] = len(df)
    except Exception as e:
        print(f"Error counting {category}: {e}")
        category_counts[category] = 0

# Print results        
print("Variant Category Counts:")
for category, count in category_counts.items():
    print(f"{category:>15}: {count:>6,} rows")

# Calculate and print total
total = sum(category_counts.values())
print(f"{'Total':>15}: {total:>6,} rows")

category: total (mapped + unmapped to (uniprot))

missense: 261500 rows (261346 + 154)

nonsense: 54675 rows (54648 + 27)

coding_indels: 103732 rows (101769 + 1963)

non_coding: 58607 rows 

other_variants: 24613 rows